In [1]:
# start

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1674374694707_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pandas as pd
from PIL import Image
import numpy as np
import io
#import pyspark
import sys
#from pyspark.context import SparkContext
#from pyspark import SparkConf
import random
import shutil
import os
from os import listdir

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType
#from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
PATH = 's3://oc-projet8-zhao-yixuan'
PATH_Data = PATH+'/sample_fruits'
PATH_Result = PATH+'/sample_fruits_result_restnet'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-projet8-zhao-yixuan
PATH_Data:   s3://oc-projet8-zhao-yixuan/sample_fruits
PATH_Result: s3://oc-projet8-zhao-yixuan/sample_fruits_result_restnet

In [5]:
# download images using Spark's binary file data source
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------+--------------------+
|                path|   modificationTime| length|             content|
+--------------------+-------------------+-------+--------------------+
|s3://oc-projet8-z...|2022-12-31 15:49:46|6457796|[FF D8 FF E1 03 F...|
|s3://oc-projet8-z...|2022-12-31 15:56:25|5704339|[FF D8 FF E1 03 F...|
|s3://oc-projet8-z...|2022-12-31 15:42:10|5612220|[FF D8 FF E1 03 4...|
|s3://oc-projet8-z...|2022-12-31 15:59:36|5601970|[FF D8 FF E1 03 F...|
|s3://oc-projet8-z...|2022-12-31 15:57:31|5443640|[FF D8 FF E1 7C 5...|
+--------------------+-------------------+-------+--------------------+
only showing top 5 rows

In [6]:
# download model
model = ResNet50(weights='imagenet',
                include_top=False,
                pooling = 'avg')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

94765736/94765736 [==============================] - 3s 0us/step

In [7]:
# model architecture
model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                        

In [8]:
# get model's weight and then broadcast the weights
model.get_weights()
bc_model_weights = sc.broadcast(model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def model_fn():

  model = ResNet50(weights='imagenet',
                  include_top=False,
                  pooling = 'avg'
                  )
  model.set_weights(bc_model_weights.value)
  return model

def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  """
  extraction de feature, normalisation, 
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1674150146645_0002/container_1674150146645_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [10]:
# We can now run featurization on our entire Spark DataFrame.
features_df = images.repartition(12).select(col("path"), featurize_udf("content").alias("features"))
features_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|                path|            features|
+--------------------+--------------------+
|s3://oc-projet8-z...|[0.115675926, 0.0...|
|s3://oc-projet8-z...|[0.45614347, 0.02...|
|s3://oc-projet8-z...|[0.20443028, 0.37...|
|s3://oc-projet8-z...|[2.59322, 0.17057...|
|s3://oc-projet8-z...|[0.05087522, 0.18...|
+--------------------+--------------------+
only showing top 5 rows

In [11]:
# dataframe shape in pyspark, 50 images and 2 columns
print((features_df.count(), len(features_df.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(50, 2)

In [12]:
# select features
df = features_df.select('features')
df.show(4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            features|
+--------------------+
|[0.9414864, 0.268...|
|[0.54442924, 0.37...|
|[0.49359757, 0.30...|
|[0.32498038, 0.01...|
+--------------------+
only showing top 4 rows

# PCA of images (dimension reduction)

In [13]:
# standarlization with StandardScaler and import PCA in pysaprk
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.ml.functions import array_to_vector

df = df.withColumn('features', array_to_vector('features'))

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df)

# transform the dataframe
df_scaled = scaler.transform(df)
df_scaled.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[0.94148641824722...|[-0.0600345929390...|
|[0.54442924261093...|[-0.4010868120115...|
|[0.49359756708145...|[-0.4447486743383...|
|[0.32498037815093...|[-0.5895823909595...|
|[0.35749319195747...|[-0.5616555129747...|
|[1.06306612491607...|[0.04439628326192...|
+--------------------+--------------------+
only showing top 6 rows

In [16]:
scaler.save(PATH + "/standard-scaler-model2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# fit columns 'scaledFeatures', 38 principal components 
pca = PCA(k=38, inputCol="scaledFeatures", outputCol="pcaFeatures")
model_pca = pca.fit(df_scaled.select('scaledFeatures'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
model_pca.save(PATH + "/pca-model2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Transfomation 
df_pca_transformed = model_pca.transform(df_scaled)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# show 3 Columns 
df_pca_transformed.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|            features|      scaledFeatures|         pcaFeatures|
+--------------------+--------------------+--------------------+
|[0.94148641824722...|[-0.0600345929390...|[23.9486617457757...|
|[0.54442924261093...|[-0.4010868120115...|[-17.904616833834...|
|[0.49359756708145...|[-0.4447486743383...|[-15.438486785575...|
|[0.32498037815093...|[-0.5895823909595...|[-12.017291947423...|
|[0.35749319195747...|[-0.5616555129747...|[-4.4839920539265...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

In [21]:
#df_pca_transformed.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_pca_transformed.write.mode("overwrite").json(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_result = spark.read.json(PATH_Result + '/*.json')
df_result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|            features|         pcaFeatures|      scaledFeatures|
+--------------------+--------------------+--------------------+
|{1, [1.0630661249...|{1, [-16.73256618...|{1, [0.0443962832...|
|{1, [0.0057828184...|{1, [4.7917858008...|{1, [-0.863757101...|
|{1, [2.0475454330...|{1, [-11.67846630...|{1, [0.8900146809...|
|{1, [0.0264240484...|{1, [28.099590943...|{1, [-0.846027318...|
|{1, [2.5932199954...|{1, [8.6854023569...|{1, [1.3587217887...|
|{1, [0.9414864182...|{1, [23.948661745...|{1, [-0.060034592...|
|{1, [0.5444292426...|{1, [-17.90461683...|{1, [-0.401086812...|
|{1, [0.4935975670...|{1, [-15.43848678...|{1, [-0.444748674...|
|{1, [0.3249803781...|{1, [-12.01729194...|{1, [-0.589582390...|
|{1, [0.3574931919...|{1, [-4.483992053...|{1, [-0.561655512...|
|{1, [0.0, 0.0, 0....|{1, [-8.201313589...|{1, [-0.868724252...|
|{1, [0.0576503910...|{1, [0.8522991609...|{1, [-0.819205455...|
|{1, [0.0132347308...|{1,